In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
import re

In [98]:
race_id_list = []
for place in range(1, 11, 1):
    for kai in range(1, 7, 1):
        for day in range(1, 13, 1):
            for r in range(1, 13, 1):
                race_id = "2021" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)

In [94]:
# テスト用
race_id_list = race_id_list[:10]

In [100]:
race_id_list

['202101010101',
 '202101010102',
 '202101010103',
 '202101010104',
 '202101010105',
 '202101010106',
 '202101010107',
 '202101010108',
 '202101010109',
 '202101010110',
 '202101010111',
 '202101010112',
 '202101010201',
 '202101010202',
 '202101010203',
 '202101010204',
 '202101010205',
 '202101010206',
 '202101010207',
 '202101010208',
 '202101010209',
 '202101010210',
 '202101010211',
 '202101010212',
 '202101010301',
 '202101010302',
 '202101010303',
 '202101010304',
 '202101010305',
 '202101010306',
 '202101010307',
 '202101010308',
 '202101010309',
 '202101010310',
 '202101010311',
 '202101010312',
 '202101010401',
 '202101010402',
 '202101010403',
 '202101010404',
 '202101010405',
 '202101010406',
 '202101010407',
 '202101010408',
 '202101010409',
 '202101010410',
 '202101010411',
 '202101010412',
 '202101010501',
 '202101010502',
 '202101010503',
 '202101010504',
 '202101010505',
 '202101010506',
 '202101010507',
 '202101010508',
 '202101010509',
 '202101010510',
 '202101010511

In [99]:
colum = ['track', 'rightORleft', 'distance','whether','situation','time']

In [ ]:
options = Options() 
options.add_argument('--headless')
options.add_argument('--disable-gpu')             
options.add_argument('--disable-extensions')       
options.add_argument('--proxy-server="direct://"') 
options.add_argument('--proxy-bypass-list=*')      
options.add_argument('--start-maximized')      
driver = Chrome(executable_path="", options=options)

target_url = "https://regist.netkeiba.com/account/?pid=login"
driver.get(target_url)
sleep(1)

USERNAME = ""
PASSWORD = ""

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[1]/input')
sleep(1)
username_input[0].send_keys(USERNAME)
sleep(1)

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[2]/input')
username_input[0].send_keys(PASSWORD)
sleep(1)

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/div[1]/input')
sleep(1)
username_input[0].click()
sleep(1)

race = {}
race_results = {}
horse_jockey_id_list = {}
for race_id in tqdm(race_id_list):
    try:
        sleep(1)
        rece = driver.get('https://db.netkeiba.com/race/' + race_id)

        # レースTable
        sleep(1)
        table = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table')
        html = table[0].get_attribute('outerHTML')
        sleep(1)
        race[race_id] = pd.read_html(html)[0]


        # 天気など
        sleep(1)
        table = driver.find_elements(By.XPATH,'//*[@id="main"]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span')
        html = table[0].get_attribute('outerHTML')

        track = str(html).replace('<span>', '').replace('&nbsp', '')[0]
        rightORleft = str(html).replace('<span>', '').replace('&nbsp', '')[1]
        distance = str(html).replace('<span>', '').replace('&nbsp', '').split(';')[0][2:]
        whether = str(html).replace('<span>', '').replace('&nbsp', '').split(';')[2][-1]
        situation = str(html).replace('<span>', '').replace('&nbsp', '').split(';')[4].split(' : ')[1]
        time = str(html).replace('<span>', '').replace('&nbsp', '').split(';')[6].split(' : ')[1].replace('</span>','')
        
        sleep(1)
        horse_table = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table')

        sleep(1)
        horse_html = horse_table[0].get_attribute('outerHTML')

        sleep(1)
        df = pd.read_html(horse_html)[0]
        forCount = len(df)

        resultListList = []
        for i in range(forCount):
            resultList = []
            
            
            resultList.append(track)
            resultList.append(rightORleft)
            resultList.append(distance)
            resultList.append(whether)
            resultList.append(situation)
            resultList.append(time)
            resultListList.append(resultList)

        race_results[race_id] = pd.DataFrame(resultListList,columns=colum)

        sleep(1)
        table = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table/tbody/tr')
        horse_jockey_id = []
        for i in range(2,len(table)+1):
            result_id = []

            # horse_id
            sleep(1)
            horse_id = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table/tbody/tr[' + str(i) +']/td[4]')
            horse_id = re.sub( r'\D+', '', horse_id[0].get_attribute('outerHTML').split('/horse')[1].split('title')[0])
            result_id.append(horse_id)

            # jockey_id
            sleep(1)
            jockey_id = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table/tbody/tr[' + str(i) +']/td[7]')
            jockey_id = re.sub( r'\D+', '', jockey_id[0].get_attribute('outerHTML').split('/recent')[1])
            result_id.append(jockey_id)

            horse_jockey_id.append(result_id)        
            
        horse_jockey_id_list[race_id] = pd.DataFrame(horse_jockey_id,columns=['horse_id', 'jockey_id'])
    except Exception:
        continue

for key in race_results:
    race_results[key].index = [key] * len(race_results[key])
results1 = pd.concat([race_results[key] for key in race_results], sort=False)
results1.to_pickle('new_results_detail_2022.pickle')

for key in race:
    race[key].index = [key] * len(race[key])
results2 = pd.concat([race[key] for key in race], sort=False)
results2.to_pickle('new_results_2022.pickle')

for key in horse_jockey_id_list:
    horse_jockey_id_list[key].index = [key] * len(horse_jockey_id_list[key])
results3 = pd.concat([horse_jockey_id_list[key] for key in horse_jockey_id_list], sort=False)
results3.to_pickle('new_results__horse_jockey_2022.pickle')

In [32]:
df1 = pd.read_pickle('')

In [33]:
df2 = pd.read_pickle('')

In [37]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True)

In [36]:
# df.to_pickle('merge_result_2021.pickle')

In [ ]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True)

In [38]:
df2 = pd.read_pickle('')

In [40]:
df1 = pd.read_pickle('')
df2 = pd.read_pickle('')
pd.concat([df1,df2])

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,...,備考,調教師,馬主,賞金(万円),track,rightORleft,distance,whether,situation,time
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,秋山稔樹,1:08.7,NaN,87.0,...,NaN,[西] 小林真也,松岡隆雄,510.0,芝,右,1200m,雨,良,09:50
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,秋山稔樹,1:08.7,NaN,87.0,...,NaN,[西] 小林真也,松岡隆雄,510.0,芝,右,1200m,雨,良,09:50
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,秋山稔樹,1:08.7,NaN,87.0,...,NaN,[西] 小林真也,松岡隆雄,510.0,芝,右,1200m,雨,良,09:50
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,秋山稔樹,1:08.7,NaN,87.0,...,NaN,[西] 小林真也,松岡隆雄,510.0,芝,右,1200m,雨,良,09:50
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,秋山稔樹,1:08.7,NaN,87.0,...,NaN,[西] 小林真也,松岡隆雄,510.0,芝,右,1200m,雨,良,09:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202209040912,14,3,3,ピンクダイヤ,牡3,55.0,浜中俊,1:27.4,3,64.0,...,出遅れ,[西] 森秀行,木村久子,NaN,ダ,右,1400m,晴,良,16:10
202209040912,14,3,3,ピンクダイヤ,牡3,55.0,浜中俊,1:27.4,3,64.0,...,出遅れ,[西] 森秀行,木村久子,NaN,ダ,右,1400m,晴,良,16:10
202209040912,14,3,3,ピンクダイヤ,牡3,55.0,浜中俊,1:27.4,3,64.0,...,出遅れ,[西] 森秀行,木村久子,NaN,ダ,右,1400m,晴,良,16:10
202209040912,14,3,3,ピンクダイヤ,牡3,55.0,浜中俊,1:27.4,3,64.0,...,出遅れ,[西] 森秀行,木村久子,NaN,ダ,右,1400m,晴,良,16:10
